In [205]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import seaborn as sns
import datetime
import sqlalchemy

In [206]:
from datetime import date
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from flask import Flask, jsonify
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, String, Text, Date, Float, func

In [137]:
engine = create_engine('sqlite:///hawaii.sqlite')

In [138]:
Base=automap_base()

In [139]:
Base.prepare(engine, reflect=True)

In [140]:
Base.classes.keys()

['station_hawaii', 'weather_measurement']

In [141]:
stations = Base.classes.station_hawaii
measurement = Base.classes.weather_measurement

In [142]:
# Heading 1: Statistical Analysis

In [143]:
stations = pd.read_sql_query("SELECT * FROM station_hawaii",engine)

In [144]:
measurement = pd.read_sql_query("SELECT * FROM weather_measurement", engine)

In [145]:
session = Session(engine)

In [146]:
measurement['date'] = measurement['date'].astype('datetime64[ns]')

In [147]:
measurement['date']

0       2010-01-01
1       2010-01-02
2       2010-01-03
3       2010-01-04
4       2010-01-07
5       2010-01-08
6       2010-01-09
7       2010-01-10
8       2010-01-11
9       2010-01-12
10      2010-01-14
11      2010-01-15
12      2010-01-16
13      2010-01-17
14      2010-01-18
15      2010-01-19
16      2010-01-20
17      2010-01-21
18      2010-01-22
19      2010-01-23
20      2010-01-24
21      2010-01-25
22      2010-01-26
23      2010-01-27
24      2010-01-28
25      2010-01-31
26      2010-02-01
27      2010-02-04
28      2010-02-05
29      2010-02-06
           ...    
18073   2017-07-17
18074   2017-07-18
18075   2017-07-19
18076   2017-07-20
18077   2017-07-21
18078   2017-07-22
18079   2017-07-23
18080   2017-07-24
18081   2017-07-25
18082   2017-07-26
18083   2017-07-27
18084   2017-07-28
18085   2017-07-29
18086   2017-07-30
18087   2017-07-31
18088   2017-08-02
18089   2017-08-03
18090   2017-08-07
18091   2017-08-08
18092   2017-08-09
18093   2017-08-10
18094   2017

In [182]:
twelve_month_data = measurement[(measurement['date'] > '2016-01-18') & (measurement['date'] < '2017-01-18')]
twelve_month_data.head(10).style.set_properties(**{'background-color': 'black','color': 'lawngreen','border-color': 'white'})

,id,station,date,prcp,tobs
2109,2110,USC00519397,2016-01-19 00:00:00,0,63
2110,2111,USC00519397,2016-01-20 00:00:00,0,68
2111,2112,USC00519397,2016-01-21 00:00:00,0,71
2112,2113,USC00519397,2016-01-22 00:00:00,0,70
2113,2114,USC00519397,2016-01-23 00:00:00,0,70
2114,2115,USC00519397,2016-01-24 00:00:00,0,70
2115,2116,USC00519397,2016-01-25 00:00:00,0.04,71
2116,2117,USC00519397,2016-01-26 00:00:00,0.16,68
2117,2118,USC00519397,2016-01-27 00:00:00,0.01,68
2118,2119,USC00519397,2016-01-28 00:00:00,0,65


In [155]:
twelve_month_prcp = twelve_month_data[['date','prcp']].set_index('date')
twelve_month_prcp.describe()

,prcp
count,2057.000000
mean,0.179266
std,0.506459
min,0.000000
25%,0.000000
50%,0.020000
75%,0.140000
max,9.640000


In [237]:
twelve_month_prcp.plot()

In [178]:
stations = pd.read_sql_query("SELECT * FROM station_hawaii", engine).set_index('id')
stations.style.set_properties(**{'background-color': 'black','color': 'lawngreen','border-color': 'white'})

,station,name,latitude,longitude,elevation
id,,,,,
1,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.817,3
2,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.802,14.6
3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.837,7
4,USC00517948,"PEARL CITY, HI US",21.3934,-157.975,11.9
5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.011,306.6
6,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.3356,-157.711,19.5
7,USC00519281,"WAIHEE 837.5, HI US",21.4517,-157.849,32.9
8,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.3152,-157.999,0.9
9,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.3331,-157.803,152.4


In [191]:
stations_available = pd.read_sql_query("SELECT *, COUNT(tobs) FROM weather_measurement GROUP BY station ORDER BY COUNT(tobs) DESC", engine)

In [192]:
stations_available.style.set_properties(**{'background-color': 'black','color': 'lawngreen','border-color': 'green'})

,id,station,date,prcp,tobs,COUNT(tobs)
0,13687,USC00519281,2017-08-18,0.06,79,2772
1,5381,USC00513117,2017-07-31,0,77,2696
2,2685,USC00519397,2017-08-23,0,81,2685
3,10915,USC00519523,2017-08-23,0.08,82,2572
4,18103,USC00516128,2017-08-23,0.45,76,2484
5,7318,USC00514830,2017-08-23,0,82,1937
6,15619,USC00511918,2015-10-30,0,64,1932
7,8001,USC00517948,2017-07-28,0.01,80,683
8,8343,USC00518838,2015-10-20,0,74,342


In [ ]:
# Station with ID USC00519281 has the highest count 

In [207]:
tobs_twelve_month = pd.read_sql_query("SELECT * FROM weather_measurement WHERE station = 'USC00519281'", engine)

In [208]:
tobs_twelve_month = tobs_twelve_month[(tobs_twelve_month['date'] > '2016-01-18') & (tobs_twelve_month['date'] < '2017-01-18')]

In [209]:
tobs_twelve_month = tobs_twelve_month[['date','tobs']].set_index('date')

In [212]:
tobs_twelve_month.plot.bar()

In [239]:
from datetime import date

In [251]:
def calc_temps(start, end):
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    start_record = date(start_date.year - 1, start_date.month, start_date.day)
    end_record = date(end_date.year - 1, end_date.month, end_date.day)
    temperature_values = session.query(weather_measurement.tobs).filter(weather_measurement.date >= start_record).filter(Measurement.date <= end_record).all()
    temps = [x[0] for x in temperature_values]
    temp_data = pd.DataFrame({'Temperature': temps})
    avg = temp_data.mean()
    max_temp = (temp_data.max() - avg)
    min_temp = (avg - temp_data.min())
    plt.bar(1,avg, width=2, yerr=[min_temp,max_temp])
    plt.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off')
    plt.title('Trip Average Temperature')
    plt.ylabel('Temperature (F)')
    plt.show()